In [14]:
pip install -U langchain langchain-core langchain-community langchain-groq python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [ ]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.tools import tool


# Load environment variables

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")


# 1. Initialize Groq LLM

llm = ChatGroq(
    model="llama-3.3-70b-versatile",  # Updated to supported model
    temperature=0
)


# 2. Simple Chain (Prompt → LLM → Output)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("user", "{input}")
])

chain = prompt | llm | StrOutputParser()

print("----- SIMPLE CHAIN OUTPUT -----")
print(chain.invoke({"input": "Explain LangChain in one sentence"}))


# 3. Tools (Function Calling)

@tool
def km_to_meters(km: float) -> float:
    """Convert kilometers to meters"""
    return km * 1000

@tool
def add_numbers(a: float, b: float) -> float:
    """Add two numbers together"""
    return a + b

@tool
def multiply_numbers(a: float, b: float) -> float:
    """Multiply two numbers together"""
    return a * b

tools = [km_to_meters, add_numbers, multiply_numbers]


# 4. Simple Tool Calling with LLM

print("\n----- TOOL CALLING WITH LLM -----")

# Bind tools to LLM
llm_with_tools = llm.bind_tools(tools)

# Create a simple prompt for tool usage
tool_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant with access to tools. Use them when needed."),
    ("human", "{input}")
])

# Create chain with tools
tool_chain = tool_prompt | llm_with_tools

# Invoke and check for tool calls
response = tool_chain.invoke({"input": "Convert 5 km to meters"})

print("\nLLM Response:")
print(response)

# Check if tools were called
if hasattr(response, 'tool_calls') and response.tool_calls:
    print("\n----- TOOL CALLS DETECTED -----")
    for tool_call in response.tool_calls:
        print(f"\nTool: {tool_call['name']}")
        print(f"Arguments: {tool_call['args']}")
        
        # Execute the tool manually
        if tool_call['name'] == 'km_to_meters':
            result = km_to_meters.invoke(tool_call['args'])
            print(f"Result: {result}")
        elif tool_call['name'] == 'add_numbers':
            result = add_numbers.invoke(tool_call['args'])
            print(f"Result: {result}")
        elif tool_call['name'] == 'multiply_numbers':
            result = multiply_numbers.invoke(tool_call['args'])
            print(f"Result: {result}")


# 5. Alternative: Direct Tool Usage Without Agent

print("\n\n----- DIRECT TOOL USAGE -----")

# Manual approach - just call the tools directly
km_value = 5
meters = km_to_meters.invoke({"km": km_value})
print(f"{km_value} km = {meters} meters")

# Chain tool calls
result = add_numbers.invoke({"a": meters, "b": 500})
print(f"{meters} + 500 = {result}")


# 6. LangGraph Approach (Modern Alternative)

print("\n\n----- USING LANGGRAPH (RECOMMENDED) -----")

try:
    from langgraph.prebuilt import create_react_agent
    
    # Create agent using LangGraph
    agent_executor = create_react_agent(llm, tools)
    
    # Run the agent
    result = agent_executor.invoke({
        "messages": [("human", "Convert 5 km to meters and then add 500 to the result")]
    })
    
    print("\nAgent Response:")
    for message in result["messages"]:
        if hasattr(message, 'content'):
            print(message.content)
            
except ImportError:
    print("LangGraph not available. Install with: pip install langgraph")
    print("\nAlternative: Use direct tool calling as shown above")


# 7. Simple Manual Agent Loop (Fallback)

print("\n\n----- MANUAL AGENT LOOP -----")

def run_manual_agent(query: str, max_iterations: int = 3):
    """Simple manual agent implementation"""
    print(f"\nQuery: {query}")
    
    messages = [
        ("system", "You are a helpful assistant. You have access to these tools: km_to_meters, add_numbers, multiply_numbers. Use them when needed."),
        ("human", query)
    ]
    
    for i in range(max_iterations):
        print(f"\n--- Iteration {i+1} ---")
        response = llm_with_tools.invoke(messages)
        
        # Check if we have tool calls
        if hasattr(response, 'tool_calls') and response.tool_calls:
            for tool_call in response.tool_calls:
                tool_name = tool_call['name']
                tool_args = tool_call['args']
                
                print(f"Calling tool: {tool_name} with args: {tool_args}")
                
                # Execute tool
                if tool_name == 'km_to_meters':
                    tool_result = km_to_meters.invoke(tool_args)
                elif tool_name == 'add_numbers':
                    tool_result = add_numbers.invoke(tool_args)
                elif tool_name == 'multiply_numbers':
                    tool_result = multiply_numbers.invoke(tool_args)
                
                print(f"Tool result: {tool_result}")
                
                # Add tool result to messages
                messages.append(("assistant", f"Tool {tool_name} returned: {tool_result}"))
        else:
            # No more tool calls, we have final answer
            print(f"\nFinal Answer: {response.content}")
            return response.content
    
    return "Max iterations reached"

# Test the manual agent
run_manual_agent("Convert 3 km to meters, then add 1000 to the result")

----- SIMPLE CHAIN OUTPUT -----
LangChain is an open-source framework that enables developers to build applications powered by large language models, providing a set of tools and libraries to simplify the process of integrating AI models into software development.

----- TOOL CALLING WITH LLM -----

LLM Response:
content='' additional_kwargs={'tool_calls': [{'id': 'rxmx6dpy7', 'function': {'arguments': '{"km":5}', 'name': 'km_to_meters'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 343, 'total_tokens': 359, 'completion_time': 0.026133916, 'completion_tokens_details': None, 'prompt_time': 0.018697911, 'prompt_tokens_details': None, 'queue_time': 0.054435139, 'total_time': 0.044831827}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_c06d5113ec', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b6098-d019-7862-8445-bdf459a5a6d1-0' tool_calls=[{'n

/tmp/ipykernel_8119/2892613163.py:120: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools)



Agent Response:
Convert 5 km to meters and then add 500 to the result

5000.0
5500.0
The result of converting 5 km to meters is 5000 meters. Then, adding 500 to the result gives 5500 meters.


----- MANUAL AGENT LOOP -----

Query: Convert 3 km to meters, then add 1000 to the result

--- Iteration 1 ---
Calling tool: km_to_meters with args: {'km': 3}
Tool result: 3000.0
Calling tool: add_numbers with args: {'a': 3000, 'b': 1000}
Tool result: 4000.0

--- Iteration 2 ---
Calling tool: km_to_meters with args: {'km': 3}
Tool result: 3000.0
Calling tool: add_numbers with args: {'a': 3000, 'b': 1000}
Tool result: 4000.0

--- Iteration 3 ---
Calling tool: km_to_meters with args: {'km': 3}
Tool result: 3000.0
Calling tool: add_numbers with args: {'a': 3000, 'b': 1000}
Tool result: 4000.0


'Max iterations reached'